# Project: **Data Jobs Salaries in August 2022 in Mexico**
____
____

## **1. Data Collection: Web scraping**
___

Author: **Daniel Eduardo López**

GitHub: **_https://github.com/DanielEduardoLopez_**

LinkedIn: **_https://www.linkedin.com/in/daniel-eduardo-lopez_**

Email: **_daniel-eduardo-lopez@outlook.com_**
____

The purpose of this notebook is to retrieve job data from the OCC's website (OCC.com.mx) through web scraping. To do so, two functions are defined: 
- **occscraper**, which scrapes the OCC website and returns the results in a Pandas dataframe, and
- **get_classid**, which returns a sample of the OCC website to allow to identify the current class IDs of the OCC website to effectively perform the web scraping.

The OCC website dynamically sets the class identifiers for its page elements. So, to perform the web scraping, first, it is strongly advised to first execute the **get_classid** function, then **inspect** what are the **current class identifiers** and finally execute the **occscraper** function to produce the desired results.


In [5]:
!pip install selenium
!pip install webdriver_manager

In [2]:
# Libraries importation
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

In [33]:
# Function to scrape job data from OCC.com.mx
def occscraper(jobs_list, number_pages, vacancy_class, jobname_class, salary_class, company_class, location_class):
    """
    This function scrapes job data from the OCC Website (occ.com.mx): Position Name, Salary, Company and Location.

    It requires 7 inputs: 
    1. jobs_list : List with the name of the Data Jobs in both English and Spanish and avoiding empty words (Python list of strings).
    2. number_pages : Number of pages to scrap from the website (Integer).
    3. vacancy_class : Class identifier for the vacancy, for instance: 'c0132 c011010' (String)
    4. jobname_class : Class identifier for the name of the position, for instance: 'c01584 c01588 c01604 c01990 c011016' (String)
    5. salary_class : Class identifier for the salary of the position, for instance: 'c01584 c01591 c01604 c01993' (String)
    6. company_class : Class identifier for the company offering the position, for instance: 'c011000' (String)
    7. location_class : Class identifier for the geographical location of the position, for instance: 'c011005 c011006' (String)

    Output: 
    1. Pandas Dataframe with the results in a tabular form from the web scraping.

    IMPORTANT NOTE: OCC Website dynamically sets the class identifiers for its page elements. So, surely the example class 
    identifiers will not produce results when running the present code in a different moment than the one when this code was 
    written and run. Thus, to RE-RUN the code, first, it is strongly advised to first execute the get_classid() function and then 
    INSPECT what are the CURRENT class identifiers to produce NEW results.

    IMPORTANT NOTE 2: This code works with Mozilla Firefox web browser.
    """

    # Setting of the base url of the OCC searcher
    base_url = "https://www.occ.com.mx/empleos/de-"
    base_page_url = "?page="

    # Creation of the corresponding url for each job from the jobs list
    jobs_url_list = list(jobs_list)
    length = len(jobs_url_list)

    for i in range(0,length):
        jobs_url_list[i] = jobs_url_list[i].strip()
        jobs_url_list[i] = jobs_url_list[i].lower()
        jobs_url_list[i] = jobs_url_list[i].replace(' ','-')
        jobs_url_list[i] = base_url + jobs_url_list[i]
        jobs_url_list[i] = jobs_url_list[i] + '/'
        #print(jobs_url_list[i])

    # Setting of the executable path in a new service instance 
    service = Service(executable_path=GeckoDriverManager().install())

    # Creation of a new instance of the Firefox driver
    driver = webdriver.Firefox(service = service)
    driver.implicitly_wait(1)

    # Creation of the list to store the data
    data = []

    # Iterations over the different jobs
    for job_url in jobs_url_list:
        
        # Start of the loop
        print('Fetching data for:', jobs_list[jobs_url_list.index(job_url)].title(), 
            ' ({} out of {})'.format(jobs_url_list.index(job_url)+1, length))
        
        # Creation of the different pages for the job
        pages_url_list = []
        for j in range(1, number_pages + 1):
            if j == 1:
                pages_url_list.append(job_url)
            else:
                pages_url_list.append(job_url + base_page_url + str(j))
            
        # Web scrapping over the different pages
        for url in pages_url_list:
            
            # Adding try tag in case urls might have a problem
            try:
                # Soup creation
                driver.get(url)
                # wait = wait(driver, 10) # Didn't work
                # salary = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, salary_class))) # Didn't work

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                # Data extraction
                vacancies = soup.find_all('div', attrs = {'class': vacancy_class})
                
                for vacancy in vacancies:
                    job = []
                    
                    try:
                        job.append(vacancy.find('h2', attrs = {'class': jobname_class}).text)
                    except:
                        job.append(np.nan) # In case there is no job name available

                    
                    try:
                        #job.append(driver.find_element(By.CLASS_NAME, salary_class))
                        job.append(vacancy.find('span', attrs = {'class': salary_class}).text)
                    except:
                        job.append(np.nan) # In case there is no salary available
                    
                    try:
                        job.append(vacancy.find('a', attrs = {'class': company_class}).text)
                    except:
                        job.append(np.nan) # In case there is no company name available

                    try:
                        job.append(vacancy.find('a', attrs = {'class': location_class}).text)
                    except:
                        job.append(np.nan) # In case there is no location available

                    data.append(job)
            
            except:
                continue
        
        # End of the urls loop
        print('Successfully retrieved data for:', jobs_list[jobs_url_list.index(job_url)].title(),
            ' ({} out of {})'.format(jobs_url_list.index(job_url) + 1, length) +'\n')

    # End of the main loop
    print('Job done!\n\n')

    # Closure of the Driver
    driver.quit()

    # Store results as a data frame
    df = pd.DataFrame(data, columns = ['Job','Salary','Company','Location'])

    return df

In [7]:
# Function to get OCC's current Class IDs
def get_classid(jobs_list):
    """
    This function retrieves a sample of the OCC's website to allow the user to identify the current Class IDs for the relevant page elements, in order to allow the
    subsequent web scraping of the page.
    
    It is important to note that OCC Website dynamically sets the class identifiers for its page elements. Thus, to effectively scrape the website, first, it is necessary 
    to the load the a sample of page source and then INSPECT what are the CURRENT class identifiers to produce results.

    IMPORTANT NOTE: This code works with Mozilla Firefox web browser.

    Input: 
    1. jobs_list : List with the name of the Data Jobs in both English and Spanish and avoiding empty words (Python list of strings).

    Ouput:
    1. Sample of the OCC's website for the first job in the jobs list.
    """
    
    # Setting of the executable path in a new service instance 
    service = Service(executable_path=GeckoDriverManager().install())

    # Creation of a new instance of the Firefox driver
    driver = webdriver.Firefox(service = service)
    
    # Setting of the base url of the OCC searcher
    base_url = "https://www.occ.com.mx/empleos/de-"
    
    # Creation of the corresponding url for each job from the jobs list
    jobs_url_list = list(jobs_list)
    length = len(jobs_url_list)

    for i in range(0,length):
        jobs_url_list[i] = jobs_url_list[i].strip()
        jobs_url_list[i] = jobs_url_list[i].lower()
        jobs_url_list[i] = jobs_url_list[i].replace(' ','-')
        jobs_url_list[i] = base_url + jobs_url_list[i]
        jobs_url_list[i] = jobs_url_list[i] + '/'
            
    # Request of the sample page source
    driver.get(jobs_url_list[0])
    html_test = driver.page_source

    # Closure of the Driver
    driver.quit()

    return html_test

In [35]:
# Entry of the Data Jobs in both English and Spanish (avoid empty words) in a Python list

#jobs_list = ["analista datos"] # Test job list

jobs_list = ["analista datos",
           "data analyst",
           "cientifico datos",
           "data scientist",
           "ingeniero datos",
           "data engineer",
           "arquitecto datos",
           "data arquitect",
           "analista negocio",
           "business analyst"]

# This list was based on: 
    # Axistalent (2020). The Ecosystem of Data Jobs - Making sense of the Data Job Market. https://www.axistalent.io/blog/the-ecosystem-of-data-jobs-making-sense-of-the-data-job-market 


In [36]:
# Number of pages to scrap
number_pages = 10

In [8]:
# Retrieval of the current class identifiers from the OCC Website
get_classid(jobs_list)

[WDM] - Downloading: 16.2kB [00:00, 281kB/s]                    
[WDM] - Downloading: 100%|██████████| 1.42M/1.42M [00:00<00:00, 4.08MB/s]


'<html class="windows desktop landscape" lang="es"><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta http-equiv="Cache-control" content="public">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale = 5.0">\n<meta name="theme-color" content="#083CAE">\n<link rel="preload" href="https://cdn-h4.occ.com.mx/fonts/hkgrotesk-semibold.woff2" as="font" type="font/woff2" crossorigin="">\n<link rel="preload" href="https://cdn-h4.occ.com.mx/fonts/hkgrotesk-regular.woff" as="font" type="font/woff" crossorigin="">\n<link href="https://cdn-h4.occ.com.mx/fonts/stylesheet.css" rel="preload stylesheet" as="style">\n<link rel="dns-prefetch icon" as="image" href="https://cdn-h4.occ.com.mx/images/common/favicon.png" type="image/png">\n<link rel="canonical" href="https://www.occ.com.mx/empleos/de-analista-datos/">\n<link rel="preload manifest" href="https://cdn-h4.occ.com.mx/pwa/manifest.json">\n<link data-react-helmet="true" rel="next" href=

In [31]:
# Entry of the OCC Website class identifiers
vacancy_class = 'c01594 c01596 c01599 c011063' # div class
jobname_class = 'c01607 c01611 c01627 c011065 c011091' # h2 class
salary_class = 'c01607 c01614 c01627 c011068' # span class 
company_class = 'c011075' # a class
location_class = 'c011080 c011081' # a class

In [37]:
# Current time
ct = datetime.datetime.now()
print("current time:", ct)

current time: 2022-08-03 21:07:46.757476


In [38]:
# Call of the web scraping function
df = occscraper(jobs_list, number_pages, vacancy_class, jobname_class, salary_class, company_class, location_class)
df

Fetching data for: Analista Datos  (1 out of 10)
Successfully retrieved data for: Analista Datos  (1 out of 10)

Fetching data for: Data Analyst  (2 out of 10)
Successfully retrieved data for: Data Analyst  (2 out of 10)

Fetching data for: Cientifico Datos  (3 out of 10)
Successfully retrieved data for: Cientifico Datos  (3 out of 10)

Fetching data for: Data Scientist  (4 out of 10)
Successfully retrieved data for: Data Scientist  (4 out of 10)

Fetching data for: Ingeniero Datos  (5 out of 10)
Successfully retrieved data for: Ingeniero Datos  (5 out of 10)

Fetching data for: Data Engineer  (6 out of 10)
Successfully retrieved data for: Data Engineer  (6 out of 10)

Fetching data for: Arquitecto Datos  (7 out of 10)
Successfully retrieved data for: Arquitecto Datos  (7 out of 10)

Fetching data for: Data Arquitect  (8 out of 10)
Successfully retrieved data for: Data Arquitect  (8 out of 10)

Fetching data for: Analista Negocio  (9 out of 10)
Successfully retrieved data for: Analista

,Job,Salary,Company,Location
0,Analista de datos,"$47,000 - $53,000 Mensual",P3 IMPULSORES ESTRATEGICOS,CDMX
1,Analista de Datos,"$35,000 Mensual",Corporativo Mek de México,Ciudad de México
2,Analista de datos,NaN,FIDELITY MARKETING S.A DE C.V,CDMX
3,Analista de datos,"$13,000 - $13,300 Mensual",Universidad Tecnológica de México,Ciudad de México
4,Analista de Datos,NaN,Laboratorios Médicos del Chopo,CDMX
...,...,...,...,...
1274,Tech Lead CX Digital Wealth,NaN,Banamex,Ciudad de México
1275,Strategy Lead Analyst,NaN,Banamex,Ciudad de México
1276,Vacante PAM - Tech Lead & Tech Owner,NaN,Banamex,Ciudad de México
1277,ComplAML KYC Analyst 2,NaN,Banamex,Ciudad de México


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Job       1279 non-null   object
 1   Salary    482 non-null    object
 2   Company   1130 non-null   object
 3   Location  1279 non-null   object
dtypes: object(4)
memory usage: 40.1+ KB


In [40]:
# Data exporting to a CSV file
df.to_csv('Dataset_Raw.csv', index=False, encoding='utf-8')